In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -U transformers==4.42.3
!pip install -U tokenizers
!pip install -U peft
!pip install -U bitsandbytes
!pip install -U datasets

#エラーが出たので、下記のバージョンにダウングレード

In [ ]:
# #tensorflowでTPUを動かす！

# %tensorflow_version 2.x
# import tensorflow as tf
# print("Tensorflow version " + tf.__version__)

# try:
#     tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
#     print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
# except ValueError:
#     raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

# tf.config.experimental_connect_to_cluster(tpu)
# tf.tpu.experimental.initialize_tpu_system(tpu)
# tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [ ]:
import os
import pandas as pd
import torch
import sklearn
import numpy as np
import time
import unicodedata

import transformers
import tokenizers
import peft
import bitsandbytes
from transformers import AutoTokenizer, BitsAndBytesConfig
from transformers import Gemma2ForSequenceClassification
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType
from peft import prepare_model_for_kbit_training
from torch.cuda.amp import autocast
from transformers import DataCollatorWithPadding
from datasets import DatasetDict, Dataset
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from transformers import Trainer, TrainingArguments
import torch.nn.functional as F
import warnings
warnings.filterwarnings('ignore')

# ライブラリーのバージョン確認
print(f"transformers: {transformers.__version__}")
print(f"tokenizers: {tokenizers.__version__}")
print(f"peft: {peft.__version__}")
# print(f"bitsandbytes: {bitsandbytes.__version__}")      # バージョン古いとエラー出る
!pip show BitsAndBytes

#演算中のメモリアウトが解決しない場合、Trueに変更
#半精度演算
torch.backends.cuda.enable_mem_efficient_sdp(False)
#フラッシュ半精度演算(よりメモリ効率重視)
torch.backends.cuda.enable_flash_sdp(False)


transformers: 4.42.3
tokenizers: 0.19.1
peft: 0.11.1
Name: bitsandbytes
Version: 0.43.1
Summary: k-bit optimizers and matrix multiplication routines.
Home-page: https://github.com/TimDettmers/bitsandbytes
Author: Tim Dettmers
Author-email: dettmers@cs.washington.edu
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: numpy, torch
Required-by: 


In [ ]:
#事前学習済みGemma2モデルの読み込みにHuggingfaceへのログインが必要

!pip install huggingface_huba
from huggingface_hub import notebook_login

notebook_login()

ERROR: Could not find a version that satisfies the requirement huggingface_huba (from versions: none)
ERROR: No matching distribution found for huggingface_huba


In [ ]:
train_data = pd.read_csv('/content/drive/Shared drives/BLM_AI開発室共有/05_Kaggle/共有/train.csv')
columns = train_data.columns
print(train_data.shape)
# Ultrafeedback_data = pd.read_csv('/content/drive/Shared drives/BLM_AI開発室共有/05_Kaggle/共有/ultrafeedback_chosen_ver2.csv')
# Ultrafeedback_data = Ultrafeedback_data[train_data.columns]

MODEL_NAME = 'google/gemma-2-9b-it'

MAX_LENGTH = 1536
BATCH_SIZE = 4
DEVICE = torch.device("cuda")
class_weights = None    #クラスの分布を考慮したいときの重要度の変数（今回は分布がほぼ均一なので、Noneに設定）
TEST_MODE = False      #動作検証用のフラグ変数
Train_half = False    #半分で訓練（精度検証）
RESUME_TRAIN = False   #モデルの再訓練のためのフラグ変数

#スモールスタート
if TEST_MODE:
    train_data = train_data.iloc[-1000:,:].reset_index(drop=True)

if RESUME_TRAIN:
    checkpoint_dir = '/content/drive/Shared drives/BLM_AI開発室共有/05_Kaggle/共有/lmsys_classification/checkpoint-7185'
    print('Resume training setting')
    # train_data = pd.concat([train_data, Ultrafeedback_data], axis=0).reset_index(drop=True)


(57477, 9)
Resume training setting


In [ ]:
#kaggle APIを使ってLlama3モデルの事前学習済みモデルの読み込みを行う（互換性を確保）

# from google.colab import files
# files.upload()

# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json
# !pip install kaggle

# import kagglehub

# # Download latest version
# path = kagglehub.model_download("metaresearch/llama-3/transformers/8b-chat-hf")

# print("Path to model files:", path)

# !cp -r {path} '/content/drive/Shared drives/BLM_AI開発室共有/05_Kaggle/共有/Llama3_basemodel'

In [ ]:
# 前処理

def process(input_str):
    stripped_str = input_str.strip('[]')
    sentences = [s.strip('"') for s in stripped_str.split('","')]
    return  ' '.join(sentences)

train = train_data.copy()
train.loc[:, 'prompt'] = train['prompt'].apply(process)
train.loc[:, 'response_a'] = train['response_a'].apply(process)
train.loc[:, 'response_b'] = train['response_b'].apply(process)


# Drop 'Null' for training
indexes = train[(train.response_a == 'null') & (train.response_b == 'null')].index
train.drop(indexes, inplace=True)
train.reset_index(inplace=True, drop=True)


# テキスト準備
train['text'] = 'User prompt: ' + train['prompt'] +  '\n\nModel A :\n' + train['response_a'] +'\n\n--------\n\nModel B:\n'  + train['response_b']
print(train['text'][0])

#正解ラベル作成
def label_to_int(row):
    if row['winner_model_a'] == 1:
        return 0
    elif row['winner_model_b'] == 1:
        return 1
    elif row['winner_tie'] == 1:
        return 2
    else:
        print('Error')
        return None

train['labels'] = train[['winner_model_a', 'winner_model_b', 'winner_tie']].apply(lambda x: label_to_int(x),axis=1)


display(train.head(5))

# if Train_half:
#     train = train[-int(len(train) * 0.5):].reset_index(drop=True)

User prompt: 'OK, does pineapple belong on a pizza? Relax and give me fun answer.'

Model A :
As an AI, I don't eat, so I don't have personal preferences. But, in the world of pizza, the pineapple topping is a hotly debated topic! Some people love the sweet and savory mix it brings to a pizza, a flavor combination made famous by the Hawaiian pizza. Others firmly believe that the sweetness of the fruit has no place on a pizza. It's a pizza puzzle that might never be solved. So, whether pineapple belongs on a pizza or not, it truly depends on your taste buds!

--------

Model B:
'Ah, the age-old culinary conundrum that has divided nations and dinner tables: does pineapple belong on a pizza? The tropical twist of pineapple on pizza, known as Hawaiian pizza, is a hotly debated topic where taste buds battle and pizza purists protest.\n\nLet\'s slice into the debate with a zest of fun:\n\n**Team Pineapple:** "Absolutely, yes! Pineapple on pizza is like a beach party in your mouth. The sweet 

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie,text,labels
0,30192,gpt-4-1106-preview,gpt-4-0613,"'OK, does pineapple belong on a pizza? Relax a...","As an AI, I don't eat, so I don't have persona...","'Ah, the age-old culinary conundrum that has d...",0,1,0,"User prompt: 'OK, does pineapple belong on a p...",1
1,53567,koala-13b,gpt-4-0613,'What is the minimal time to get them? 1 day o...,The marriage license in California is issued i...,The minimal time to get a marriage license and...,1,0,0,User prompt: 'What is the minimal time to get ...,0
2,481524,gpt-4-0314,gpt-3.5-turbo-0613,What's your favorite food?,"Oh, my favorite food is sushi, uwu! I just lov...","As an AI, I don't eat, but if I had to pick a ...",1,0,0,User prompt: What's your favorite food?\n\nMod...,0
3,604575,pplx-70b-online,gpt-3.5-turbo-1106,'did hitler support israel',"No, Adolf Hitler did not support Israel. In fa...",'Adolf Hitler and the Nazi regime did not supp...,1,0,0,User prompt: 'did hitler support israel'\n\nMo...,0
4,738614,mpt-30b-chat,gpt4all-13b-snoozy,'Tell me more about tic-tac-toe strategies.',There are several strategies that can help you...,"'Tic-tac-toe is a simple game, but there are s...",0,1,0,User prompt: 'Tell me more about tic-tac-toe s...,1


In [ ]:
%%time
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'
tokenizer.add_eos_token = True

data = train.copy()

# Utility function giving token length
def get_token_lengths(texts):
    # tokenize and receive input_ids for reach text
    input_ids = tokenizer(texts.tolist(), return_tensors='np')['input_ids']
    # return length of inputs_ids for each text
    return [len(t) for t in input_ids]

def get_inputs(texts):
    # tokenize and receive input_ids for reach text
    input_ids = tokenizer(texts.tolist(), return_tensors='np')['input_ids']
    # return length of inputs_ids for each text
    return input_ids

prompt_inputs = get_inputs(data['prompt'])
res_a_inputs = get_inputs(data['response_a'])
res_b_inputs = get_inputs(data['response_b'])
data['total_tokens'] = get_token_lengths(data['text'])


#max_lengthを超える文章データのインデックスを取得
indicies = data[data['total_tokens']>MAX_LENGTH].index

#トークンを調整して、デコード処理、データフレームに調整した文章を戻す処理

from tqdm import tqdm
def assign_tokens(data, indicies, max_length):

    for ind in tqdm(indicies):

        # トークン長取得
        len_p = len(prompt_inputs[ind])
        len_a = len(res_a_inputs[ind])
        len_b = len(res_b_inputs[ind])
        total = len_p + len_a + len_b

        # 割り当てるトークン数をそれぞれ設定
        p_assign = int((len_p / total) * max_length)
        a_assign = int((len_a / total) * max_length)
        b_assign = max_length - (p_assign + a_assign)

        # トークンを先頭からスライス
        prompt_inputs[ind] = prompt_inputs[ind][:p_assign]
        res_a_inputs[ind] = res_a_inputs[ind][:a_assign]
        res_b_inputs[ind] = res_b_inputs[ind][:b_assign]

        # デコードしてデータに格納
        data.loc[ind, 'prompt'] = tokenizer.decode(prompt_inputs[ind], skip_special_tokens=True)
        data.loc[ind, 'response_a'] = tokenizer.decode(res_a_inputs[ind], skip_special_tokens=True)
        data.loc[ind, 'response_b'] = tokenizer.decode(res_b_inputs[ind], skip_special_tokens=True)

    return data

train = assign_tokens(data, indicies, MAX_LENGTH)

train['text'] = 'User prompt: ' + train['prompt'] +  '\n\nModel A :\n' + train['response_a'] +'\n\n--------\n\nModel B:\n'  + train['response_b']

dataset = Dataset.from_pandas(train[['text', 'labels']])

del data

100%|██████████| 142/142 [00:00<00:00, 491.66it/s]


CPU times: user 32.8 s, sys: 1.19 s, total: 34 s
Wall time: 8 s


In [ ]:
%%time
data = pd.DataFrame()

#add_prefix_space＝各単語の前にスペースを追加するかどうか
#Trueだと多言語に対する単語分割の性能が向上するが、メモリ使用量増加する
#Trueするとトークン数に若干影響でる（元の文の先頭にスペースがないものや日本語や中国語は特にトークン数増える）＝＞悪影響だったかもしれないです
# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME,
#                                         #   add_prefix_space=True
#                                           )

#パディングとEOS(end of sequence(文章の終端))のトークンを同じものにそろえる処理
#これにより、語彙サイズの減少によるメモリ効率の向上とモデル性能の向上の可能性を確保している
# tokenizer.pad_token_id = tokenizer.eos_token_id
# tokenizer.pad_token = tokenizer.eos_token

def data_preprocesing(row):
    return tokenizer(row['text'], truncation=True, padding='max_length',
                       max_length=MAX_LENGTH)

tokenized_data = dataset.map(data_preprocesing, batched=True, remove_columns=['text'])
tokenized_data.set_format("torch")

#トークナイザーの保存
tokenizer.save_pretrained('/content/drive/Shared drives/BLM_AI開発室共有/05_Kaggle/共有/gemma2-tokenizer')

Map:   0%|          | 0/7539 [00:00<?, ? examples/s]

CPU times: user 19.6 s, sys: 314 ms, total: 20 s
Wall time: 6.21 s


('/content/drive/Shared drives/BLM_AI開発室共有/05_Kaggle/共有/gemma2-tokenizer/tokenizer_config.json',
 '/content/drive/Shared drives/BLM_AI開発室共有/05_Kaggle/共有/gemma2-tokenizer/special_tokens_map.json',
 '/content/drive/Shared drives/BLM_AI開発室共有/05_Kaggle/共有/gemma2-tokenizer/tokenizer.model',
 '/content/drive/Shared drives/BLM_AI開発室共有/05_Kaggle/共有/gemma2-tokenizer/added_tokens.json',
 '/content/drive/Shared drives/BLM_AI開発室共有/05_Kaggle/共有/gemma2-tokenizer/tokenizer.json')

In [ ]:
tokenized_data.shape

(7539, 3)

In [ ]:
# BitsAndBytes configuration
# 8bit量子化を適用
bnb_config =  BitsAndBytesConfig(
    load_in_8bit=True,
    # load_in_4bit = True,
    # bnb_8bit_compute_dtype=torch.float16,
    # bnb_8bit_use_double_quant=False
    )

# Load base model on GPU 0
device0 = torch.device('cuda:0')

if RESUME_TRAIN == False:
    base_model_0 = Gemma2ForSequenceClassification.from_pretrained(
        MODEL_NAME,
        num_labels=3,
        torch_dtype=torch.float32,
        quantization_config=bnb_config,
        device_map='cuda:0',
        # output_hidden_states=True         # 特徴次元の出力を残すかどうか
        )

    base_model_0.config.pad_token_id = tokenizer.pad_token_id    #モデルのパディングidをトークナイザーとそろえる
    base_model_0.config.use_cache = False                        #過去の計算結果を保持するかどうか（メモリ効率のためFalseに設定）
    base_model_0.config.pretraining_tp = 1                       #使用GPUを一つに限定

In [ ]:
# LoRa configuration

peft_config = LoraConfig(
    r=32,                                     #LoRAのランク（小さいほどメモリ消費抑える）
    lora_alpha=64,                            #LoRAアダプターの学習率
    lora_dropout=0.05,
    bias='none',
    task_type=TaskType.SEQ_CLS,               #モデルのタスクタイプ（序列分類タスク）を指定
    target_modules=['o_proj', "v_proj",
                    "q_proj", "k_proj",
                    'down_proj', 'gate_proj', 'up_proj'
                     ]                              # モデル内のどの層を更新するかを制御。
        )

# Get peft
if RESUME_TRAIN == False:
    model_0 = prepare_model_for_kbit_training(base_model_0)
    model_0 = get_peft_model(model_0, peft_config).to(device0)


In [ ]:
#明示的にメモリ解放を行う
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
#バッチ処理のための関数（適用すると、バッチごとにデータ長をそろえる）
#なくても動作はしそうですが、おまじない的に入れました

from transformers import DataCollatorWithPadding

collate_fn = DataCollatorWithPadding(tokenizer=tokenizer)


In [ ]:
# #訓練可能なパラメーターの情報取得（loraなどの設定後にちゃんと設定情報がモデルに踏襲されているかを確認）
# MODEL_LAYERS_ROWS = []
# TRAINABLE_PARAMS = []
# N_TRAINABLE_PARAMS = 0

# for name, param in base_model_0.named_parameters():
#     # Layer Parameter Count
#     n_parameters = int(torch.prod(torch.tensor(param.shape)))
#     # Only Trainable Layers
#     if param.requires_grad:
#         # Add Layer Information
#         MODEL_LAYERS_ROWS.append({
#             'param': n_parameters,
#             'name': name,
#             'dtype': param.data.dtype,
#         })
#         # Append Trainable Parameter
#         TRAINABLE_PARAMS.append({ 'params': param })
#         # Add Number Of Trainable Parameters"
#         N_TRAINABLE_PARAMS += n_parameters

# display(pd.DataFrame(MODEL_LAYERS_ROWS))

# print(f"""
# ===============================
# N_TRAINABLE_PARAMS: {N_TRAINABLE_PARAMS:,}
# N_TRAINABLE_LAYERS: {len(TRAINABLE_PARAMS)}
# ===============================
# """)

In [ ]:
#評価関数の定義（検証データを訓練時に渡すのであれば、必要）
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import EvalPrediction

# def compute_metrics(evaluations):
#     predictions, labels = evaluations
#     if isinstance(predictions, list):
#         for i, pred in enumerate(predictions):
#             print(f"Prediction {i} shape: {np.array(pred).shape}")

#     predictions = np.argmax(predictions, axis=1)
#     return {'balanced_accuracy' : balanced_accuracy_score(predictions, labels),
#     'accuracy':accuracy_score(predictions,labels)}

def compute_metrics(p: EvalPrediction):
    predictions, labels = p.predictions, p.label_ids

    # predictionsがタプルである場合、その内容を確認
    if isinstance(predictions, tuple):
        predictions = predictions[0]

    # predictionsがlogitsの場合、argmaxを取ってクラスラベルに変換
    if predictions.ndim == 2:
        predictions = np.argmax(predictions, axis=1)

    # 精度を計算
    accuracy = accuracy_score(labels, predictions)

    return {
        'accuracy': accuracy
    }


In [ ]:
class CustomTrainer(Trainer):
    def __init__(self, *args, class_weights=None, **kwargs):
        super().__init__(*args, **kwargs)
        if class_weights is not None:
            self.class_weights = torch.tensor(class_weights,
            dtype=torch.float32).to(self.args.device)
        else:
            self.class_weights = None

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels").long()

        outputs = model(**inputs)

        logits = outputs.get('logits')

        if self.class_weights is not None:
            loss = F.cross_entropy(logits, labels, weight=self.class_weights)
        else:
            loss = F.cross_entropy(logits, labels)

        return (loss, outputs) if return_outputs else loss


In [ ]:
training_args = TrainingArguments(
        output_dir = '/content/drive/Shared drives/BLM_AI開発室共有/05_Kaggle/共有/lmsys_classification',
        learning_rate = 5e-5,
        per_device_train_batch_size = BATCH_SIZE,
        gradient_accumulation_steps = 2,
        per_device_eval_batch_size = BATCH_SIZE,
        num_train_epochs = 1,
        logging_steps=10,                 # 途中経過の出力頻度
        weight_decay = 0.01,
        save_strategy="steps",             # チェックポイントをステップごとに保存
        # evaluation_strategy="steps",     # 検証をstepsごとに評価
        # load_best_model_at_end = True,   # 検証データに対する最も優秀なモデルの重みを保存
        report_to="none",                  # wandbに保存しないように調整
        save_steps=100,                    # 保存するステップ数を調整
        fp16=True,                         # AMPを有効にする
        # gradient_checkpointing=True  # Gradient Checkpointingを有効にする
        )

In [ ]:
# if not RESUME_TRAIN:
#     train_test_dataset = tokenized_data.train_test_split(test_size=0.1)
#     train_test_dataset['test']
#     train_test_dataset['test'].to_csv('/content/drive/Shared drives/BLM_AI開発室共有/05_Kaggle/共有/検証用データ.csv')

In [ ]:
tokenized_data

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 7539
})

In [ ]:
if not RESUME_TRAIN:
    trainer = CustomTrainer(
        model = model_0,
        args = training_args,
        train_dataset = tokenized_data,
        # eval_dataset = train_test_dataset['test'],
        data_collator = collate_fn,
        compute_metrics = compute_metrics,
        class_weights=class_weights,
    )

    train_result = trainer.train()
    trainer.save_model('/content/drive/Shared drives/BLM_AI開発室共有/05_Kaggle/共有/Gemma2_fine-tuning')

テンソル型でモデルの重みパラメーターの保存  
kaggleでモデルの重みを事前学習済みモデルから更新するときに必要

In [ ]:
#torchで保存（kaggleとの互換性を確保）

# device0 = torch.device('cuda:0')

# check_point_dir = '/content/drive/Shared drives/BLM_AI開発室共有/05_Kaggle/共有/lmsys_classification/checkpoint-1797'

# tokenizer = AutoTokenizer.from_pretrained('/content/drive/Shared drives/BLM_AI開発室共有/05_Kaggle/共有/gemma2-tokenizer')
# tokenizer.pad_token = tokenizer.eos_token
# tokenizer.padding_side = 'right'
# tokenizer.add_eos_token = True

# # 8bit量子化を適用
# bnb_config =  BitsAndBytesConfig(
#     load_in_8bit=True,
#     # load_in_4bit = True,
#     # bnb_8bit_compute_dtype=torch.float16,
#     # bnb_8bit_use_double_quant=False
#     )

# base_model_0 = Gemma2ForSequenceClassification.from_pretrained(
#         check_point_dir,
#         num_labels=3,
#         torch_dtype=torch.float32,
#         quantization_config=bnb_config,
#         device_map='cuda:0',
#         # output_hidden_states=True         # 特徴次元の出力を残すかどうか
#         )

# base_model_0.config.pad_token_id = tokenizer.pad_token_id    #モデルのパディングidをトークナイザーとそろえる
# base_model_0.config.use_cache = False                        #過去の計算結果を保持するかどうか（メモリ効率のためFalseに設定）
# base_model_0.config.pretraining_tp = 1                       #使用GPUを一つに限定


# peft_config = LoraConfig(
#     r=4,                                     #LoRAのランク（小さいほどメモリ消費抑える）
#     lora_alpha=8,                            #LoRAアダプターの学習率
#     lora_dropout=0.05,
#     bias='none',
#     task_type=TaskType.SEQ_CLS,               #モデルのタスクタイプ（序列分類タスク）を指定
#     target_modules=['o_proj', "v_proj",
#                     #  "q_proj", "k_proj"
#                      ]                              # モデル内のどの層を更新するかを制御。
#         )

# # Get peft
# model_0 = prepare_model_for_kbit_training(base_model_0)
# model_0 = get_peft_model(model_0, peft_config).to(device0)

# model = model_0.cpu()
# torch.save(dict([(k,v) for k, v in model.named_parameters() if v.requires_grad]), '/content/drive/Shared drives/BLM_AI開発室共有/05_Kaggle/共有/gemma2_finetuned_model.pth')

# model = base_model_0.cpu()
# torch.save(model.state_dict(), '/content/drive/Shared drives/BLM_AI開発室共有/05_Kaggle/共有/gemma2_finetuned_model.pth')


In [ ]:
# from sklearn.metrics import accuracy_score, confusion_matrix
# from sklearn.metrics import balanced_accuracy_score, classification_report

# def get_performance_metrics(test_df):
#     y_test = test_df.label
#     y_pred = test_df.predictions

#     print("Classification Report:")
#     print(classification_report(y_test, y_pred))

#     print("Balanced Accuracy Score:", balanced_accuracy_score(y_test, y_pred))
#     print("Accuracy Score:", accuracy_score(y_test, y_pred))

# def generate_predictions(model,df_test):
#     sentences = df_test.text.tolist()
#     batch_size = 8
#     all_outputs = []

#     for i in range(0, len(sentences), batch_size):

#         batch_sentences = sentences[i:i + batch_size]

#         inputs = tokenizer(batch_sentences, return_tensors="pt",
#         padding=True, truncation=True, max_length=1536)

#         inputs = {k: v.to('cuda' if torch.cuda.is_available() else 'cpu')
#         for k, v in inputs.items()}

#         with torch.no_grad():
#             outputs = model(**inputs)
#             all_outputs.append(outputs['logits'])

#     final_outputs = torch.cat(all_outputs, dim=0)
#     df_test['predictions']=final_outputs.argmax(axis=1).cpu().numpy()

# test_df = train.sample(100).reset_index(drop=True)
# generate_predictions(model_0,test_df)
# get_performance_metrics(test_df)

# Resume Train

In [ ]:

if RESUME_TRAIN:
    checkpoint_dir = '/content/drive/Shared drives/BLM_AI開発室共有/05_Kaggle/共有/lmsys_classification/checkpoint-7185'

    saved_model = Gemma2ForSequenceClassification.from_pretrained(
        checkpoint_dir,
        num_labels=3,
        torch_dtype=torch.float32,
        quantization_config=bnb_config,
        device_map='cuda:0',)


    saved_model.config.pad_token_id = tokenizer.pad_token_id    #モデルのパディングidをトークナイザーとそろえる
    saved_model.config.use_cache = False                        #過去の計算結果を保持するかどうか（メモリ効率のためFalseに設定）
    saved_model.config.pretraining_tp = 1                       #使用GPUを一つに限定

    # Get peft
    saved_model = prepare_model_for_kbit_training(saved_model)
    saved_model = get_peft_model(saved_model, peft_config).to(device0)
    saved_model.train()


    # saved_model = PeftModel.from_pretrained(saved_model, checkpoint_dir)

    trainer = CustomTrainer(
        model = saved_model,
        args = training_args,
        train_dataset = tokenized_data,
        # eval_dataset = train_test_dataset['test'],
        data_collator = collate_fn,
        compute_metrics = compute_metrics,
        class_weights=class_weights,
    )


    train_result = trainer.train(resume_from_checkpoint=True, ignore_data_skip=True)
    trainer.save_model('/content/drive/Shared drives/BLM_AI開発室共有/05_Kaggle/共有/lmsys_classification/finetuned-Gemma2-0718')

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of Gemma2ForSequenceClassification were not initialized from the model checkpoint at google/gemma-2-9b-it and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
